In [9]:
import requests
import string
from pprint import pprint
import re
import os
import tempfile
from bs4 import BeautifulSoup
from collections import Counter, OrderedDict
from operator import itemgetter
import numpy as np
hookRegex = re.compile(r'\[.*\]')
tireRegex = re.compile(r' -')
chars_to_remove = string.punctuation.replace('-', '') + '—„“»«'
print(chars_to_remove)
api_url = 'http://api.genius.com'
base_url = 'http://genius.com'
headers = {'Authorization': 'Bearer ' + open('./.bearer_token.txt', 'r').read().rstrip('\n')}

!"#$%&'()*+,./:;<=>?@[\]^_`{|}~—„“»«


In [12]:
def remove_unnecessary(textfile):
    return tireRegex.sub('', hookRegex.sub('',textfile).translate(str.maketrans('','',chars_to_remove)))

class Artist(object):
    artist_id = ''
    songlist = []
    artist_name = ''
    def __init__(self, name):
        self.name = name
        print("Finding the ID of an artist " + self.name)
        search_url = api_url + '/search'
        data = {'q': self.name}
        response = requests.get(search_url, params=data, headers=headers)
        self.artist_id = response.json()['response']['hits'][0]['result']['primary_artist']['api_path']
        print("ID of an artist " + self.name + " is " + self.artist_id[9:])
        self.artist_name = response.json()['response']['hits'][0]['result']['primary_artist']['name']
        with open('./artists_id_list.txt', 'r+') as textfile:
            if self.artist_id in textfile.read():
                pass
            else:
                textfile.write(textfile.read() + '\n' + self.artist_name + "  -  " + self.artist_id)
        list_of_songs = []
        print("Getting the list of songs")
        for page in range(1,4):
            res = requests.get(api_url + self.artist_id + '/songs?sort=popularity&per_page=50&page=' + str(page), headers=headers)
            print("Going through page " + str(page))
            for song in res.json()['response']['songs']:
                if ("Ft." in song['full_title']) | ("Remix" in song['full_title']):
                    print(song['full_title'] + " is a feat, skipping")
                else:
                    song_url = song['url']
                    list_of_songs.append(song_url)
        self.songlist = list_of_songs

    
#     def get_artist_id(self):
#         print("Finding the ID of an artist " + self.name)
#         search_url = api_url + '/search'
#         data = {'q': self.name}
#         response = requests.get(search_url, params=data, headers=headers)
#         self.artist_id = response.json()['response']['hits'][0]['result']['primary_artist']['api_path']
#         print("ID of an artist " + self.name + " is " + self.artist_id[9:])
#         self.artist_name = response.json()['response']['hits'][0]['result']['primary_artist']['name']
#         with open('./artists_id_list.txt', 'r+') as textfile:
#             if self.artist_id in textfile.read():
#                 pass
#             else:
#                 textfile.write(textfile.read() + '\n' + self.artist_name + "  -  " + self.artist_id)
#         return
#     #artists/143013/songs?sort=popularity&per_page=50
#     #artists/143013/songs?sort=popularity&per_page=50&page=3
#     def get_songlist(self):
#         list_of_songs = []
#         print("Getting the list of songs")
#         for page in range(1,4):
#             res = requests.get(api_url + self.artist_id + '/songs?sort=popularity&per_page=50&page=' + str(page), headers=headers)
#             print("Going through page " + str(page))
#             for song in res.json()['response']['songs']:
# #                 print(song['full_title'])
#                 if ("Ft." in song['full_title']) | ("Remix" in song['full_title']):
#                     print(song['full_title'] + " is a feat, skipping")
#                 else:
#                     song_url = song['url']
#                     list_of_songs.append(song_url)
#         self.songlist = list_of_songs
#         return
    
    
    def perform_task(self):
        with open("./lyrics/" + self.artist_name.replace(' ', '_') + '_lyrics.txt', 'w') as lyricsfile:
            for song in self.songlist:
                print(str(self.songlist.index(song)) + " Now scraping " + song[19:])
                page_url = song
                res = requests.get(page_url, headers=headers)
                res.text
                page = requests.get(page_url)
                html = BeautifulSoup(page.text, 'html.parser')
                lyrics = html.find(class_='lyrics').get_text()
                lyricsfile.write(remove_unnecessary(lyrics.lower()))
        return
    
#     def perform_task(self, write_id=False):
#         self.get_artist_id()
#         self.get_songlist()
#         self.write_lyrics_from_path()
#         return


In [13]:
with open('./artists_source.txt', 'r') as id_list:
    for artist in id_list.readlines():
        the_artist = Artist(artist.rstrip("\n"))
        the_artist.perform_task(write_id=True)

Finding the ID of an artist oxxxymiron
ID of an artist oxxxymiron is 24422
Getting the list of songs
Going through page 1
12/04/15: Oxxxymiron vs. Johnyboy by Versus Battle (Ft. Johnyboy & Oxxxymiron) is a feat, skipping
Fata Morgana by Markul (Ft. Oxxxymiron) is a feat, skipping
Безумие (Madness) [Remix] by ЛСП (LSP) (Ft. Oxxxymiron) is a feat, skipping
Пора возвращаться домой (PVD) by БИ-2 (BI-2) (Ft. Oxxxymiron) is a feat, skipping
Больше Бена (Bolshe Bena) by Oxxxymiron (Ft. Охра (Ohra)) is a feat, skipping
13/08/2017: Oxxxymiron vs. Слава КПСС by Versus Battle (Ft. Oxxxymiron & Слава КПСС (Slava KPSS)) is a feat, skipping
Darkside by Oxxxymiron (Ft. MadChild) is a feat, skipping
Imperial by Porchy (Ft. ЛСП (LSP) & Oxxxymiron) is a feat, skipping
Бездыханным (Breathless) by Жак-Энтони (Jacques-Anthony) (Ft. Oxxxymiron) is a feat, skipping
Машина Прогресса (Progress Machine) by Ка-тет (Ka-tet) (Ft. Oxxxymiron) is a feat, skipping
Привет со дна  (Hello from the bottom) by Oxxxymiron 

Going through page 2
Going through page 3
0 Now scraping Makulatura-reprise-lyrics
1 Now scraping Makulatura-nate-diaz-lyrics
2 Now scraping Makulatura-how-the-dead-love-lyrics
3 Now scraping Makulatura-in-a-fire-lyrics
4 Now scraping Makulatura-singularity-lyrics
5 Now scraping Makulatura-30-30-let-kak-myortv-lyrics
6 Now scraping Makulatura-the-smartest-people-alive-lyrics
7 Now scraping Makulatura-limb-lyrics
8 Now scraping Makulatura-wrists-lyrics
9 Now scraping Makulatura-together-annotated
10 Now scraping Makulatura-30-dead-for-30-years-lyrics
11 Now scraping Makulatura-letuchiy-gollandec-lyrics
12 Now scraping Makulatura-nojevoe-lyrics
13 Now scraping Makulatura-alzheimer-lyrics
14 Now scraping Makulatura-gift-horse-lyrics
15 Now scraping Makulatura-samiy-otstoyniy-den-lyrics
16 Now scraping Makulatura-the-all-universe-lyrics
17 Now scraping Makulatura-beach-lyrics
18 Now scraping Makulatura-walter-lyrics
19 Now scraping Makulatura-stage-lyrics
20 Now scraping Makulatura-the-fat

62 Now scraping Guf-naugad-lyrics
63 Now scraping Centr-porosya-lyrics
64 Now scraping Guf-metropolitan-mail-lyrics
65 Now scraping Guf-dlya-nee-lyrics
66 Now scraping Guf-dno-lyrics
67 Now scraping Guf-and-rigos-long-way-lyrics
68 Now scraping Centr-po-zhesti-lyrics
69 Now scraping Guf-and-rigos-inhale-lyrics
70 Now scraping Guf-excursion-lyrics
71 Now scraping Centr-77-zhara-77-lyrics
72 Now scraping Guf-and-rigos-parasite-lyrics
73 Now scraping Guf-ka-kalpak-lyrics
74 Now scraping Guf-and-rigos-renegade-lyrics
75 Now scraping Guf-novenkiy-lyrics
76 Now scraping Guf-and-rigos-low-profile-lyrics
77 Now scraping Guf-and-rigos-bezvilazno-lyrics
78 Now scraping Centr-peremotka-lyrics
79 Now scraping Guf-sam-skit-lyrics
80 Now scraping Guf-skit-ot-sanka-lyrics
81 Now scraping Centr-slaydi-lyrics
82 Now scraping Centr-kak-nastroenie-lyrics
83 Now scraping Guf-hobby-lyrics
84 Now scraping Guf-and-rigos-industry-lyrics
85 Now scraping Guf-and-rigos-pali-jk-lyrics
86 Now scraping Centr-sosedi

Going through page 3
ВСРАТОЧКА МОЯ    (VSRATOCHKA MOYA) by Слава КПСС (Slava KPSS) (Ft. MC Jiglypuf) is a feat, skipping
Русская музыка для тренировок (Russkaya muzyka dlya trenirovok) by Слава КПСС (Slava KPSS) (Ft. мц лучник (mc luchnik)) is a feat, skipping
MC by СД (SD aka Sadist) (Ft. Букер (Booker) & Слава КПСС (Slava KPSS)) is a feat, skipping
Стикербой (Stickerboy) by Хан Замай (Khan Zamay) & Слава КПСС (Slava KPSS) (Ft. Stephan Pie) is a feat, skipping
Весна, мать, пришла (Vesna mat prishla) by Слава КПСС (Slava KPSS) (Ft. Скудно (Skudno)) is a feat, skipping
Бытиё есть Панчлайн (Existence is Punchline) by Хан Замай (Khan Zamay) & Слава КПСС (Slava KPSS) (Ft. Вандальский (Vandalskiy)) is a feat, skipping
Вне игры (Out of the Game Long Mix) by Хан Замай (Khan Zamay) & Слава КПСС (Slava KPSS) (Ft. MORRALLES, Sasha, СД (SD aka Sadist) & Zoo In Space) is a feat, skipping
В палёном Антихайпе (Fake Antihype) by СД (SD aka Sadist) (Ft. DK (D.K. Inc.) & Слава КПСС (Slava KPSS)) is a f

Going through page 3
Мы бухаем до утра (We Drink All Night) by Piggy Bang (Ft. Den Hawk, ЛСП (LSP) & Young Bumer) is a feat, skipping
Как бы не так (I Don't Think So) by Piggy Bang (Ft. Den Hawk, ЛСП (LSP), Young Bumer & Young Sever) is a feat, skipping
Хиппи (Hippie) [Aes Genius Remix] by ЛСП (LSP) is a feat, skipping
ЧУДО-УДО (Chudo-Udo) by Саша Скул (Sasha Skul) (Ft. ЛСП (LSP)) is a feat, skipping
Привет (Hello) by Lottery Billz (Ft. джетвилланс (jetvillains)) is a feat, skipping
Каждый день (Everyday) [Remix] by Piggy Bang (Ft. ItsKostoBaby, ЛСП (LSP) & Young Bumer) is a feat, skipping
Соседка (Female Neighbor) by Piggy Bang (Ft. Den Hawk, Ващекинг (VascheKing) & Young Bumer) is a feat, skipping
Пиксель (Pixel) by Paul Pain (Ft. ЛСП (LSP)) is a feat, skipping
Красиво (Nice) by Piggy Bang (Ft. ЛСП (LSP) & Young Bumer) is a feat, skipping
N… by Piggy Bang (Ft. Den Hawk, ЛСП (LSP) & Young Bumer) is a feat, skipping
Весна (Spring) by Piggy Bang (Ft. Den Hawk, ЛСП (LSP) & Young Bumer) i

...

Going through page 2
Наше лето (Nashe leto) by Баста (Basta) (Ft. МакSим (MakSim)) is a feat, skipping
Виски и кокаин (Viski i kokain) by Баста (Basta) (Ft. Нервы (Nervy) & Смоки Мо (Smoky Mo)) is a feat, skipping
Через тернии к звездам (Through hardship...) by NEL (Ft. Баста (Basta)) is a feat, skipping
Обернись (Turn around) by Баста (Basta) (Ft. Город 312 (Gorod 312)) is a feat, skipping
Каменные цветы (Stone flowers ) by Баста (Basta) & Смоки Мо (Smoky Mo) (Ft. Елена Ваенга (Elena Vaenga)) is a feat, skipping
Космос (Cosmos) by Баста (Basta) (Ft. Чаруша (Charusha)) is a feat, skipping
Ум (Um) by Баста (Basta) (Ft. UM) is a feat, skipping
Я поднимаюсь над землей (Ya podnimayus' nad zemley) by Баста (Basta) (Ft. Алена Омаргалиева (Alena Omargaliyeva)) is a feat, skipping
Один (Odin) by Баста (Basta) (Ft. Смоки Мо (Smoky Mo)) is a feat, skipping
С надеждой на крылья (With hope for wings) by Баста (Basta) (Ft. Нервы (Nervy)) is a feat, skipping
Финальный матч (Final'nyy match) by Баста

Going through page 2
Трафик (Traffic) by Centr (Ft. Смоки Мо (Smoky Mo)) is a feat, skipping
Не на экспорт (Ne na export) by Centr (Ft. TAHDEM Foundation) is a feat, skipping
В норме (V norme) by Centr (Ft. Словетский (Slovetskii)) is a feat, skipping
Качели. часть 2 (Kacheli. part 2) by Centr (Ft. Ноггано (Noggano)) is a feat, skipping
Клещи (Kleshi) by Slim (Ft. Каспийский Груз (KG)) is a feat, skipping
Беркуты (Berkuty) by Slim (Ft. Барбитурный (Barbiturny) & Расколъ (Raskol 666)) is a feat, skipping
Далеко (Daleko) by Centr (Ft. А-Студио (A’Studio)) is a feat, skipping
Железное Небо (Zheleznoe Nebo) by Centr (Ft. 25/17, Принцип (Princip) & Рома Жиган (Roma Jigan)) is a feat, skipping
Не не (Ne ne) by Каспийский Груз (KG) (Ft. Slim) is a feat, skipping
Двигай (Dvigay) by Slim (Ft. Daffy (Russia)) is a feat, skipping
Ночь (Night) by Centr (Ft. Тати (Tati)) is a feat, skipping
Легенды (Legendi) by Centr (Ft. 5 Плюх (Five P)) is a feat, skipping
Летняя (Summer) by Slim (Ft. Daffy (Russ

1 Now scraping Schokk-htm-lyrics
2 Now scraping Schokk-rubchinsky-sweater-lyrics
3 Now scraping Schokk-round-lyrics
4 Now scraping Schokk-amen-lyrics
5 Now scraping Schokk-misanthrope-lyrics
6 Now scraping Schokk-ponedelnik-lyrics
7 Now scraping Schokk-tak-nado-lyrics
8 Now scraping Schokk-intro-lyrics
9 Now scraping Schokk-tmvt-lyrics
10 Now scraping Schokk-shpana-lyrics
11 Now scraping Schokk-questions-lyrics
12 Now scraping Schokk-bamberg-lyrics
13 Now scraping Schokk-pink-wine-lyrics
14 Now scraping Schokk-your-life-lyrics
15 Now scraping Schokk-toni-montano-lyrics
16 Now scraping Schokk-kandagach-skit-lyrics
17 Now scraping Schokk-benz-lyrics
18 Now scraping Schokk-ii-walk-ii-lyrics
19 Now scraping Schokk-iv-lyrics
20 Now scraping Schokk-rapelectroschokk-ii-lyrics
21 Now scraping Schokk-we-throw-stones-lyrics
22 Now scraping Schokk-intro-sbd-lyrics
23 Now scraping Schokk-lubov-kapitana-lyrics
24 Now scraping Schokk-sad-lyrics
25 Now scraping Schokk-outro-lyrics
26 Now scraping Sch

Going through page 3
23/11/13: Yanix vs. Galat by Versus Battle (Ft. Galat & Yanix) is a feat, skipping
Пей до дна (bottoms up) by Yanix (Ft. Bonus B & GidraGidra) is a feat, skipping
НПНБ (NPNB) by Мезза (Mezza) (Ft. Yanix) is a feat, skipping
Всё и сразу (All At Once) [Remix] by Yanix (Ft. ЛСП (LSP)) is a feat, skipping
Без Тормозов by D.masta (Ft. Booguy & Yanix) is a feat, skipping
Черный пояс (black belt) by Yanix (Ft. Мезза (Mezza)) is a feat, skipping
Еще еще (more and more) by Yanix (Ft. Чаян Фамали (Chayan Famaly)) is a feat, skipping
Супер индиго (super indigo) by Yanix (Ft. Hash Tag) is a feat, skipping
Холодный Жар (cold fever) by Yanix (Ft. Guts Out) is a feat, skipping
Моим Псам (For my Dogs) by Yanix (Ft. Bonus B, D.masta & GidraGidra) is a feat, skipping
Не один (not alone) by Yanix (Ft. Basic Boy) is a feat, skipping
0 Now scraping Yanix-hypin-lyrics
1 Now scraping Yanix-poppin-lyrics
2 Now scraping Yanix-and-thomas-mraz-i-want-to-you-lyrics
3 Now scraping Yanix-and-th

71 Now scraping Anacondaz-kak-tank-lyrics
72 Now scraping Anacondaz-doma-posizhu-lyrics
73 Now scraping Anacondaz-album-version-chestnyy-obmen-lyrics
Finding the ID of an artist карандаш
ID of an artist карандаш is 664698
Getting the list of songs
Going through page 1
Тату (Tattoo) by Тимати (Timati) (Ft. Джиган (Geegun), Карандаш (Karandash), L’One, Крэк (Rayon) & Варчун (Varchun)) is a feat, skipping
Tissot by Карандаш (Karandash) (Ft. Ёлка (Yolka)) is a feat, skipping
Сделай меня плохим (Make Me Bad) by Карандаш (Karandash) (Ft. Shaman (Russia)) is a feat, skipping
Камеры (Cameras) by Карандаш (Karandash) (Ft. Звонкий (Zvonkiy)) is a feat, skipping
Это нормально (Eto Normal'no) (RMX) by Карандаш (Karandash) (Ft. Anacondaz, Крип-а-Крип (Crip-a-Crip), Дабл (Dabl), Dime (Nonamerz), Fuze, Giga, Lenin, Марсель (Marsel), Нигатив (Nigativ), PLC, Сэт (Sette) & Варчун (Varchun)) is a feat, skipping
Королева (The Queen) by Карандаш (Karandash) (Ft. Звонкий (Zvonkiy)) is a feat, skipping
Рэп к

In [18]:
def get_percentage(words, total):
    return round(words*100/total, 2)

def ord_dict(input_dict):
    return OrderedDict(sorted(input_dict.items(), key = itemgetter(1), reverse = True))

class Lyric_file(object): 
    the_text = ''
    the_list = []
    the_counter = {}
    the_total = 0
    def __init__(self, location):
        self.location = location
        self.the_text = open(self.location, 'r').read()
        self.the_list = self.the_text.split()
        self.the_counter = Counter(self.the_list)
        self.the_total = len(self.the_list)
        
    def get_most_common(self, number):
        return self.the_counter.most_common(number)
    
    def get_most_common_percentage(self, number):
        result_list = []
        for word in self.get_most_common(number):
            result_list.append((word[0], get_percentage(word[1], len(self.the_list))))
        return result_list
    
    def get_percentage_of_word(self, word):
        return get_percentage(self.the_counter[word],len(self.the_list))
    
    def get_vocab_size(self):
        return len(self.the_list)/len(self.the_counter)
    

In [14]:
with open('./lyrics/all_lyrics_together.txt', 'w') as total_file:
    for lyrics in os.listdir('./lyrics/'):
        total_file.write(open('./lyrics/' + lyrics, 'r').read())


In [32]:
total_lyric = Lyric_file('./lyrics/all_lyrics_together.txt')
total_counter = total_lyric.the_counter
def get_unique_words(total_words, artist_count):
    shared_items = set(total_words.items()) & set(artist_count.items())
    return shared_items

In [33]:
oxy = Lyric_file('./lyrics/Oxxxymiron_lyrics.txt')
pprint(get_unique_words(total_counter, oxy.the_counter))


{('1000', 2),
 ('1001', 1),
 ('20000', 1),
 ('26', 2),
 ('38', 1),
 ('44', 1),
 ('80-ые', 1),
 ('88', 1),
 ('airmax', 1),
 ('american', 1),
 ('antwoord', 1),
 ('apartment', 1),
 ('assassins', 1),
 ('aмур', 1),
 ('backpacker', 1),
 ('barking', 1),
 ('baronу', 1),
 ('beckton', 1),
 ('bird', 1),
 ('bits', 1),
 ('bong', 3),
 ('bredren', 1),
 ('buff', 1),
 ('bullа', 1),
 ('burn', 1),
 ('butters', 1),
 ('canning', 1),
 ('ceo', 1),
 ('chat', 1),
 ('checkу', 1),
 ('citibanka', 1),
 ('cobain', 1),
 ('council-estate', 1),
 ('creed', 1),
 ('cubase', 1),
 ('cutty', 1),
 ('cкалите', 1),
 ('cредь', 1),
 ('disneyland', 1),
 ('diy', 1),
 ('dooma', 1),
 ('download', 1),
 ('drakea', 1),
 ('dronу', 1),
 ('dub-', 1),
 ('ejayе', 1),
 ('emsiom', 1),
 ('ends', 1),
 ('fam', 1),
 ('freshness', 1),
 ('gash', 1),
 ('greenparka', 1),
 ('gully', 1),
 ('gummy', 1),
 ('gutter', 1),
 ('gyeah', 1),
 ('ham', 2),
 ('having', 1),
 ('haze', 1),
 ('heineken', 1),
 ('hits', 2),
 ('hummerы', 1),
 ('innit', 1),
 ('j2', 1),
 (

In [37]:
artist_dict = {}
for artist in os.listdir('./lyrics/'):
    obj = Lyric_file('./lyrics/' + artist)
    artist_dict[artist] = len(get_unique_words(total_counter, obj.the_counter))*100/len(obj.the_counter)
pprint(ord_dict(artist_dict))

{'all_lyrics_together.txt': 100.0,
 'SidxRam_lyrics.txt': 34.341147938561036,
 'ATL_lyrics.txt': 32.27814569536424,
 'Noize_MC_lyrics.txt': 32.1009918845807,
 'Хаски_(Husky)_lyrics.txt': 31.4678681527152,
 'Кровосток_(Krovostok)_lyrics.txt': 31.14720394736842,
 'Oxxxymiron_lyrics.txt': 29.102247628946344,
 'Anacondaz_lyrics.txt': 28.818957829022807,
 'Ноггано_(Noggano)_lyrics.txt': 27.121396670726757,
 'макулатура_(makulatura)_lyrics.txt': 26.760233918128655,
 'Schokk_lyrics.txt': 26.43824516726473,
 'Каста_(Kasta)_lyrics.txt': 25.429288057109783,
 'СД_(SD_aka_Sadist)_lyrics.txt': 22.44871794871795,
 'Face_lyrics.txt': 22.20922068804119,
 'Рем_Дигга_(Rem_Digga)_lyrics.txt': 21.575178997613364,
 'Паша_Техник_(Pasha_Technique)_lyrics.txt': 21.236925875397908,
 'Скриптонит_(Scriptonite)_lyrics.txt': 21.212735753396878,
 'ЛСП_(LSP)_lyrics.txt': 21.072118480360594,
 'Big_Russian_Boss_lyrics.txt': 20.605263157894736,
 'Карандаш_(Karandash)_lyrics.txt': 20.560747663551403,
 'Pharaoh_lyrics.tx